In [2]:
import requests
from bs4 import BeautifulSoup
import pandas

In [84]:
def get_page(page):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
        # 'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.fifaindex.com/',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Sec-GPC': '1',
        # Requests doesn't support trailers
        # 'TE': 'trailers',
    }

    params = {
        'order_by': 'overallrating',
        'order': '0',
        'page': str(page)
    }
    print(f"Getting page {page}... ")
    response = requests.get('https://www.fifaindex.com/fr/players/fifa23/', params=params, headers=headers)
    print("OK")
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    table = soup.find("table", {"class":"table-players"})

    joueurs = table.find("tbody").find_all("tr", {"data-playerid": True})

    master_liste = []
    for j in joueurs:
        joueur_data = {}
        joueur_data["nom"] = j.find("td", {"data-title": "Nom"}).find("a").text
        joueur_data["nationalité"] = j.find("td", {"data-title": "Nationalité"}).find("a")["title"]
        joueur_data["rating"] = int(j.find("td", {"data-title": "GEN / POT"}).find("span").text)
        joueur_data["age"] = int(j.find("td", {"data-title": "Age"}).text)
        master_liste.append(joueur_data)
        
    return master_liste

In [85]:
data_joueurs = []
for i in range(1, 614):
    l = get_page(i)
    data_joueurs.extend(l)

Getting page 1... 
OK
Getting page 2... 
OK
Getting page 3... 
OK
Getting page 4... 
OK
Getting page 5... 
OK
Getting page 6... 
OK
Getting page 7... 
OK
Getting page 8... 
OK
Getting page 9... 
OK
Getting page 10... 
OK
Getting page 11... 
OK
Getting page 12... 
OK
Getting page 13... 
OK
Getting page 14... 
OK
Getting page 15... 
OK
Getting page 16... 
OK
Getting page 17... 
OK
Getting page 18... 
OK
Getting page 19... 
OK
Getting page 20... 
OK
Getting page 21... 
OK
Getting page 22... 
OK
Getting page 23... 
OK
Getting page 24... 
OK
Getting page 25... 
OK
Getting page 26... 
OK
Getting page 27... 
OK
Getting page 28... 
OK
Getting page 29... 
OK
Getting page 30... 
OK
Getting page 31... 
OK
Getting page 32... 
OK
Getting page 33... 
OK
Getting page 34... 
OK
Getting page 35... 
OK
Getting page 36... 
OK
Getting page 37... 
OK
Getting page 38... 
OK
Getting page 39... 
OK
Getting page 40... 
OK
Getting page 41... 
OK
Getting page 42... 
OK
Getting page 43... 
OK
Getting page 44... 


OK
Getting page 347... 
OK
Getting page 348... 
OK
Getting page 349... 
OK
Getting page 350... 
OK
Getting page 351... 
OK
Getting page 352... 
OK
Getting page 353... 
OK
Getting page 354... 
OK
Getting page 355... 
OK
Getting page 356... 
OK
Getting page 357... 
OK
Getting page 358... 
OK
Getting page 359... 
OK
Getting page 360... 
OK
Getting page 361... 
OK
Getting page 362... 
OK
Getting page 363... 
OK
Getting page 364... 
OK
Getting page 365... 
OK
Getting page 366... 
OK
Getting page 367... 
OK
Getting page 368... 
OK
Getting page 369... 
OK
Getting page 370... 
OK
Getting page 371... 
OK
Getting page 372... 
OK
Getting page 373... 
OK
Getting page 374... 
OK
Getting page 375... 
OK
Getting page 376... 
OK
Getting page 377... 
OK
Getting page 378... 
OK
Getting page 379... 
OK
Getting page 380... 
OK
Getting page 381... 
OK
Getting page 382... 
OK
Getting page 383... 
OK
Getting page 384... 
OK
Getting page 385... 
OK
Getting page 386... 
OK
Getting page 387... 
OK
Getting page 

In [88]:
df = pandas.DataFrame(data_joueurs)

In [90]:
df.to_csv("data_joueurs.csv", index=False)

In [19]:
request = requests.get("https://fr.wikipedia.org/wiki/Effectif_des_%C3%A9quipes_%C3%A0_la_Coupe_du_monde_de_football_2022")

In [20]:
html = request.text
soup = BeautifulSoup(html, "html.parser")

In [21]:
l = soup.find("li", {"id": "toc-Effectifs"}).find_all("li", {"class": "sidebar-toc-level-3"})
liste_equipes = [e.find("a")["href"][1:] for e in l]

In [26]:
liste_finale = []
for equipe in liste_equipes:
    retour = {}
    retour["nom"] = equipe
    
    retour["gardiens"] = []
    retour["défenseurs"] = []
    retour["milieux"] = []
    retour["attaquants"] = []
    

    if equipe == "États-Unis" or equipe == "Canada":
        table = soup.select(f"span#{equipe}")[0].find_next("table").find("tbody").find("table").find("tbody")

        for e in table.find_all("tr")[1:]:
            td = e.find_all("td")
            if td[1].text.strip() == "G":
                retour["gardiens"].append(td[2].find("a").text.strip())
            elif td[1].text.strip() == "D":
                retour["défenseurs"].append(td[2].find("a").text.strip())
            elif td[1].text.strip() == "M":
                retour["milieux"].append(td[2].find("a").text.strip())
            elif td[1].text.strip() == "A":
                retour["attaquants"].append(td[2].find("a").text.strip())
                
    else :
        table = soup.select(f"span#{equipe}")[0].find_next("table")
        gardiens = table.find_all("tr", {"bgcolor" : "#FFECCE"})
        defenseur = table.find_all("tr", {"bgcolor" : "#E7FAEC"})
        milieu = table.find_all("tr", {"bgcolor" : "#DFEDFD"})
        attaquant = table.find_all("tr", {"bgcolor" : "#FFD2D6"})
        

        for g in gardiens:
            retour["gardiens"].append(g.find_all("td")[1].text.strip())

        for g in defenseur:
            retour["défenseurs"].append(g.find_all("td")[1].text.strip())

        for g in milieu:
            retour["milieux"].append(g.find_all("td")[1].text.strip())
            
        for g in attaquant:
            retour["attaquants"].append(g.find_all("td")[1].text.strip())
            
    liste_finale.append(retour)

In [27]:
df = pandas.DataFrame(liste_finale)

In [31]:
df

,nom,gardiens,défenseurs,milieux,attaquants
0,Qatar,"[Saad Al Sheeb, Yousef Hassan, Meshaal Barsham]","[Pedro Miguel, Abdelkarim Hassan, Tarek Salman...","[Mohammed Waad, Abdulaziz Hatem, Ali Assadalla...","[Ahmed Alaaeldin, Mohammed Muntari, Hassan Al ..."
1,Équateur,"[Hernán Galíndez, Moisés Ramírez, Alexander Do...","[Félix Torres, Piero Hincapié, Robert Arboleda...","[José Cifuentes, Carlos Gruezo, Ayrton Preciad...","[Michael Estrada, Enner Valencia, Djorkaeff Re..."
2,Sénégal,"[Seny Dieng, Alfred Gomis, Édouard Mendy]","[Fodé Ballo-Touré, Pape Abou Cissé, Abdou Dial...","[Pathé Ciss, Krépin Diatta, Idrissa Gueye, Pap...","[Boulaye Dia, Famara Diédhiou, Bamba Dieng, Ni..."
3,Pays-Bas,"[Remko Pasveer, Justin Bijlow, Andries Noppert]","[Jurriën Timber, Matthijs de Ligt, Virgil van ...","[Steven Berghuis, Davy Klaassen, Marten de Roo...","[Steven Bergwijn, Cody Gakpo, Luuk de Jong, Me..."
4,Angleterre,"[Jordan Pickford, Nick Pope, Aaron Ramsdale]","[Kyle Walker, Luke Shaw, John Stones, Harry Ma...","[Declan Rice, Jordan Henderson, Kalvin Phillip...","[Jack Grealish, Harry Kane, Raheem Sterling, M..."
5,Iran,"[Alireza Beiranvand, Payam Niazmand, Amir Abed...","[Sadegh Moharrami, Ehsan Hajsafi, Shojae Khali...","[Saeid Ezatolahi, Alireza Jahanbakhsh, Vahid A...","[Mehdi Taremi, Karim Ansarifard, Sardar Azmoun]"
6,États-Unis,"[Matt Turner, Ethan Horvath, Sean Johnson]","[Sergiño Dest, Walker Zimmerman, Antonee Robin...","[Tyler Adams, Yunus Musah, Weston McKennie, Lu...","[Giovanni Reyna, Jesús Ferreira, Christian Pul..."
7,Pays_de_Galles,"[Wayne Hennessey, Danny Ward, Adam Davies]","[Chris Gunter, Neco Williams, Ben Davies, Chri...","[Joe Allen, Harry Wilson, Aaron Ramsey, Joe Mo...","[Brennan Johnson, Gareth Bale, Kieffer Moore, ..."
8,Argentine,"[Franco Armani, Gerónimo Rulli, Emiliano Martí...","[Juan Foyth, Nicolás Tagliafico, Gonzalo Monti...","[Leandro Paredes, Rodrigo De Paul, Exequiel Pa...","[Julián Álvarez, Lionel Messi, Ángel Di María,..."
9,Arabie_saoudite,"[Mohammed Al Rubaie, Mohammed Al-Owais, Nawaf ...","[Sultan Al-Ghanam, Abdullah Madu, Abdulelah Al...","[Salman Al-Faraj, Abdulellah Al-Malki, Abdulla...","[Firas Al-Buraikan, Salem al-Dossari, Hattan B..."


In [30]:
df.to_csv("data_équipes.csv", sep =';', index=False)